# Model serving & deployment testing

## Abstract

This notebook serves an Sklean's pipeline with a Linear Regression, as it was the best performing model so far.


Tthe model has already been trained on the full data available (_the original file `train.csv`_) during the [training phase](https://github.com/aliz-ai/ml-spec-2022/blob/demo2/Demo%20%232/notebooks/3.%20Training%20experiments.ipynb).
<br>The model has been serialized in Cloud Storage in the __modelling_serializations__ folder.
<br>The model folder path is: `gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220608_150907/to_deploy`.

Let's deploy this model and test it with an online prediction request feeding some sample input data taken from the original file `test.csv`.

## Serving the model

### Create endpoint

In [1]:
!gcloud ai endpoints create --region="europe-west3" --display-name="demo2_blackfriday"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [3559675491994566656]...
........done.
Created Vertex AI endpoint: projects/53911330556/locations/europe-west3/endpoints/3855890521587187712.


In [2]:
!gcloud ai endpoints list --region="europe-west3" --filter=display_name="demo2_blackfriday"

ENDPOINT_ID          DISPLAY_NAME
3855890521587187712  demo2_blackfriday


Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


In [3]:
ENDPOINT_ID = 3855890521587187712

In the following, we use the obtained `ENDPOINT_ID`.

### Create model

The model artifact's filename must exactly match `model.joblib`, if Joblib is used. There are other available formats, see [documentation](https://cloud.google.com/vertex-ai/docs/training/exporting-model-artifacts).

In [4]:
DATETIME = "20220608_193211"
HYPERTUNED_MODEL = f"gs://aliz-ml-spec-2022/demo-2/modelling_serializations/{DATETIME}/n_jobs=-1.joblib"
MODEL_PATH_TO_DEPLOY = f"gs://aliz-ml-spec-2022/demo-2/modelling_serializations/{DATETIME}/to_deploy"
!gsutil cp $HYPERTUNED_MODEL $MODEL_PATH_TO_DEPLOY/model.joblib

Copying gs://aliz-ml-spec-2022/demo-2/modelling_serializations/20220608_193211/n_jobs=-1.joblib [Content-Type=application/octet-stream]...
/ [0 files][    0.0 B/176.6 KiB]                                                
/ [1 files][176.6 KiB/176.6 KiB]                                                

Operation completed over 1 objects/176.6 KiB.                                    


We use the `europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest` pre-built container for prediction. 

In [5]:
!gcloud ai models upload --region="europe-west3" --display-name="demo2_blackfriday" --container-image-uri="europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest" --artifact-uri=$MODEL_PATH_TO_DEPLOY

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [8171361510421954560]...
.....done.


In [7]:
!gcloud ai models list --region="europe-west3"

MODEL_ID             DISPLAY_NAME
4772337861384994816  demo2_blackfriday


Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


In [8]:
MODEL_ID = 4772337861384994816

### Deploy model

Deployment might take a while.

In [9]:
!gcloud ai endpoints deploy-model $ENDPOINT_ID --region=europe-west3 --model=$MODEL_ID --display-name="demo2_blackfriday" --machine-type=n1-standard-2 --min-replica-count=1 --max-replica-count=1 --traffic-split=0=100

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
Waiting for operation [7405749573768970240]...
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [14]:
DEPLOYED_MODEL_ID = 2472581748542668800

#### Predict using Gcloud

In [10]:
%%writefile request.json
{
    "instances": [
        [1000004, "P00128942", "M", "46-50", 7, "B", 2, 1, "1", "11.0", "nan"],
        [1000009, "P00113442", "M", "26-35", 17, "C", 0, 0, "3", "5.0", "nan"],
        [1000010, "P00288442", "F", "36-45", 1, "B", 4, 1, "5", "14.0", "nan"]
    ]
}

Overwriting request.json


In [11]:
!gcloud ai endpoints predict $ENDPOINT_ID --region=europe-west3 --json-request=request.json

[16106.12595763068, 11403.39117999512, 7028.360205575746]


Using endpoint [https://europe-west3-prediction-aiplatform.googleapis.com/]


#### Predict using Python

In [12]:
from google.cloud import aiplatform

def predict(project: str, location: str, instances: list, endpoint: str):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    return prediction

predict(
    project="aliz-ml-spec-2022-submission",
    location="europe-west3",
    instances=[
      [1000004, "P00128942", "M", "46-50", 7, "B", 2, 1, "1", "11.0", "nan"],
      [1000009, "P00113442", "M", "26-35", 17, "C", 0, 0, "3", "5.0", "nan"],
      [1000010, "P00288442", "F", "36-45", 1, "B", 4, 1, "5", "14.0", "nan"]
    ],
    endpoint=str(ENDPOINT_ID)
)

Prediction(predictions=[16106.12595763068, 11403.39117999512, 7028.360205575746], deployed_model_id='2472581748542668800', explanations=None)

The deployed model works just fine, we can get predictions. The predictions for each method are, of course, the same.

#### Cleanup

You can also undeploy the model, delete the model and endpoint from the Vertex AI Dashboard. However, we present the cleaning with code.

In [15]:
!gcloud ai endpoints undeploy-model $ENDPOINT_ID --region="europe-west3" --deployed-model-id=$DEPLOYED_MODEL_ID

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]


In [ ]:
!gcloud ai endpoints delete $ENDPOINT_ID --region="europe-west3"

In [17]:
!gcloud ai models delete $MODEL_ID --region="europe-west3"

Using endpoint [https://europe-west3-aiplatform.googleapis.com/]
